## Python Peephole Optimizations

Peephole optimizations refer to a certain class of optimization strategies Python employs during any compilation phases.

#### Constant Expressions

Let's see how Python reduces constant expressions for optimization purposes:

In [17]:
def my_func():
    a = 24 * 60
    b = (1, 2) * 5
    c = 'abc' * 3
    d = 'ab' * 11
    e = 'the quick brown fox' * 10
    f = [1, 2] * 5


In [18]:
my_func.__code__.co_consts

(None,
 1440,
 (1, 2, 1, 2, 1, 2, 1, 2, 1, 2),
 'abcabcabc',
 'ababababababababababab',
 'the quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown fox',
 1,
 2,
 5)

As you can see in the example above, `24 * 60` was pre-calculated and cached as a constant (`1440`).

Similarly, `(1, 2) * 5` was cached as `(1, 2, 1, 2, 1, 2, 1, 2, 1, 2)` and `'abc' * 3` was cached as `abcabcabc`.

On the other hand, note how `'the quick brown fox' * 10` was **not** pre-calculated (too long).

Similarly `[1, 2] * 5` was not pre-calculated either since a list is *mutable*, and hence not a *constant*.

#### Membership Tests

In membership testing, optimizations are applied as can be seen below:

In [28]:
def my_func():
    if e in [1, 2, 3]:
        pass

In [29]:
my_func.__code__.co_consts

(None, (1, 2, 3))

As you can see, the mutable list `[1, 2, 3]` was converted to an immutable tuple. 

It is OK to do this here, since we are testing membership of the list **at that point in time**, hence it is safe to convert it to a tuple, which is more efficient than testing membership of a list.

In the same way, set membership will be converted to frozen set membership:

In [30]:
def my_func():
    if e in {1, 2, 3}:
        pass

In [31]:
my_func.__code__.co_consts

(None, frozenset({1, 2, 3}))

In general, when you are writing your code, if you can use **set** membership testing, prefer that over a list or tuple - it is quite a bit more efficient.

Let's do a small quick (and dirty) benchmark of this:

In [32]:
import string
import time 

char_list = list(string.ascii_letters)
char_tuple = tuple(string.ascii_letters)
char_set = set(string.ascii_letters)

print(char_list)
print()
print(char_tuple)
print()
print(char_set)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

('a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z')

{'t', 'X', 'Z', 'd', 'y', 'a', 'o', 'D', 'N', 'g', 'j', 'z', 'e', 'C', 'I', 'G', 's', 'P', 'x', 'w', 'K', 'Y', 'L', 'T', 'M', 'k', 'F', 'J', 'v', 'V', 'r', 'h', 'm', 'A', 'u', 'c', 'i', 'O', 'Q', 'R', 'b', 'B', 'H', 'f', 'U', 'W', 'p', 'S', 'E', 'q', 'n', 'l'}


In [33]:
def membership_test(n, container):
    for i in range(n):
        if 'p' in container:
            pass

In [34]:
start = time.perf_counter()
membership_test(10000000, char_list)
end = time.perf_counter()
print('list membership: ', end-start)

list membership:  2.267070921000027


In [35]:
start = time.perf_counter()
membership_test(10000000, char_tuple)
end = time.perf_counter()
print('tuple membership: ', end-start)

tuple membership:  2.241360952999969


In [36]:
start = time.perf_counter()
membership_test(10000000, char_set)
end = time.perf_counter()
print('set membership: ', end-start)

set membership:  0.3199037159999989


As you can see, set membership tests run quite a bit faster - which is not surprising since they are basically dictionary-like objects, so hash maps are used for looking up an item to determine membership.